In [1]:

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from datetime import datetime, timedelta, time


#   .set("spark.dynamicAllocation.shuffleTracking.enabled", "false") \
#.set("spark.kubernetes.authenticate.driver.serviceAccountName", "spark-ml-executor") \

#       .set("spark.driver.memory", "1G") \
#       .set("spark.driver.extraClassPath", "/opt/conda/lib/python3.8/site-packages/pyspark/jars/hadoop-aws-3.2.0.jar:/opt/conda/lib/python3.8/site-packages/pyspark/jars/aws-java-sdk-bundle-1.11.888.jar") \
#       .set("spark.executor.extraClassPath", "/opt/conda/lib/python3.8/site-packages/pyspark/jars/hadoop-aws-3.2.0.jar:/opt/conda/lib/python3.8/site-packages/pyspark/jars/aws-java-sdk-bundle-1.11.888.jar") \
#       .set("spark.kubernetes.executor.podTemplateFile", "/podTemplate.yml") \



def get_spark_session(app_name: str, conf: SparkConf):
    conf.setMaster("k8s://https://kubernetes.default.svc.cluster.local")
    conf \
      .set("spark.kubernetes.namespace", "spark-ml") \
      .set("spark.kubernetes.container.image", "itayb/spark:3.1.1-hadoop-3.2.0-aws") \
      .set("spark.executor.instances", "2") \
      .set("spark.executor.memory", "2g") \
      .set("spark.executor.cores", "2") \
      .set("spark.driver.port", "2222") \
      .set("spark.driver.blockManager.port", "7777") \
      .set("spark.driver.host", "jupyter.recs.svc.cluster.local") \
      .set("spark.driver.bindAddress", "0.0.0.0") \
      .set("spark.ui.port", "4040") \
      .set("spark.network.timeout", "240") \
      .set("spark.hadoop.fs.s3a.endpoint", "localstack.kube-system.svc.cluster.local:4566") \
      .set("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
      .set("spark.hadoop.fs.s3a.path.style.access", "true") \
      .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
      .set("spark.hadoop.com.amazonaws.services.s3.enableV4", "true") \
      .set("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider")

    # print(swan_spark_conf.toDebugString()) #Instance of SparkConf with options set by the extension

    return SparkSession.builder.config(conf=conf).getOrCreate()


def dates_list(start_date, end_date):
    start_date = datetime.combine(start_date, time())
    end_date = datetime.combine(end_date, time())
    if start_date <= end_date:
        return [(start_date + timedelta(days=x)).strftime("%Y-%m-%d") for x in range(0, (end_date-start_date).days + 1)]
    return list()